# the perform a mcmc sampling for Table2 1993 QS  fe_13

In [1]:
myIon = 'Fe XIII'

In [2]:
import numpy as np
import pickle
import matplotlib.pyplot as plt
import pymc

In [3]:
qtconsole

In [4]:
matplotlib qt

## for the monte-carlo markov-chain bayesian inference model of pymc developed by Chris Fonnesbeck, Anand Patil, David Huard, John Salvatier

## pymc is an earlier version of pymc3 that is now also available

In [6]:
with open('tab2_1993_qs_fe_13_model.pkl', 'rb') as inpt:
    box = pickle.load(inpt)

In [7]:
box.SpecData['filename']

'/data2/svn/data/serts-eunis/serts/table2-1993-QS_fe_13.txt'

### a plot of intensity/Gofnt(T) - the usual EM loci plot

In [8]:
plt.figure()
box.emPlot(vs='d')
plt.text(1.e+7, 2.e+26, myIon, fontsize=16)
plt.tight_layout()

In [48]:
plt.title('fe_13, T = %10.2e'%(box.Temperature[0]))

In [9]:
nDens = box.EDensity.size
print(' # of density values %5i'%(nDens))

 # of density values   241


In [10]:
densGuess = 6.e+8
Dindex = int(np.argmin(np.abs(box.EDensity - densGuess)))
' Dindex = %5i'%(Dindex)

' Dindex =   111'

### by setting the temperature by its index and the log of the EM, we can predict the spectrum

In [11]:
box.emSetIndices(Dindex)
print('density set to %10.2e '%(box.EDensity[Dindex]))

density set to   5.96e+08 


In [12]:
emGuess = 1.e+27
emLog = np.log10(emGuess)

In [13]:
box.emSetIndices(Dindex)

In [14]:
box.emSet(emLog)

### get predicted intensities from initial guess

In [15]:
box.predict()

In [16]:
box.predictPrint(outfile='predictPrintGuess.txt')

   111     5.96e+08     1.78e+06     1.00e+27
 -------------------------------------------------
  iwvl    IonS        wvl        Int       Pred   Int/Pred        chi
               wvl lvl1 lvl2                lower -                upper lineIdx predLine contribution
 -------------------------------------------------
     0   fe_13    311.574   6.13e+00   3.38e+00      1.816      2.022
         fe_13
           311.547    2   12        3s2 3p2 3P1.0 - 3s 3p3 3P2.0            29     0   0.999
 -------------------------------------------------
 -------------------------------------------------
     1   fe_13    312.171   1.78e+01   1.87e+01      0.950      1.058
         fe_13
           312.174    2   11        3s2 3p2 3P1.0 - 3s 3p3 3P1.0            24     0   1.000
 -------------------------------------------------
 -------------------------------------------------
     2   fe_13    312.907   7.34e+00   7.22e+00      1.017      1.132
         fe_13
           312.868    2   10      

## begin set up for mc sampling

In [17]:
em0 = pymc.Normal('em0', mu=emLog, tau=1./0.3, value=emLog)
d0 = pymc.DiscreteUniform('d0', 0, nDens - 1, value=Dindex)

### get the intensities to be used by pymc

In [18]:
nobs = len(box.match)
intensity = np.zeros(nobs,'float64')
for iwvl in range(nobs):
    intensity[iwvl] = box.match[iwvl]['obsIntensity']

In [19]:
@pymc.deterministic
def predictMc1(info = box, id0 = d0, em0 = em0):
    '''
    this version is to be used with pymc for bayesian modeling
    to predict the intensities of the observed lines from an emission measure
    and a temperature
    '''
    #
    predicted = np.zeros(len(info.match), 'float64')
    for iwvl, amatch in enumerate(info.match):
#            pred = np.sum(amatch['intensitySum']*self.Em)
        predicted[iwvl] += amatch['intensitySum'][id0]*10.**em0
        info.match[iwvl]['predicted'] = predicted[iwvl]
#            self.Predicted[iwvl] = pred
#            self.Tmax[iwvl] = amatch['tmax']
    return predicted


In [20]:
box.WghtFactor = 0.2

In [21]:
obs = pymc.Normal('obs', predictMc1, 1./(box.WghtFactor*intensity)**2, value=intensity, observed=True, size=nobs)

In [22]:
model = pymc.MCMC([obs, box, em0, d0,  predictMc1], db='pickle')

In [23]:
iter1 = 100000

In [24]:
model.sample(iter=iter1, burn_till_tuned=True)

 [-------------------113%--------------------] 119158 of 105000 complete in 15.5 sec

### the statistics from the model run are stored in model.stats()

In [25]:
myQuantiles = [2.5, 5., 16., 25, 50, 75, 84., 95., 97.5]

In [26]:
model.write_csv('modelMc1.csv',variables=['d0','em0'],quantiles = myQuantiles)

In [27]:
model.stats().keys()

dict_keys(['em0', 'predictMc1', 'd0'])

In [31]:
allstats = model.stats()
allstats.keys()

dict_keys(['em0', 'predictMc1', 'd0'])

### pickle allStats for later analysis

In [32]:
with open('allstats.pkl','wb') as outpt:
    pickle.dump(allStats,outpt)

## run next cell for just doing analysis ##

In [13]:
with open('allstats.pkl','rb') as inpt:
    allStats = pickle.load(inpt)

In [33]:
allStats['d0']

{'n': 100000,
 'standard deviation': 4.531176290501177,
 'mean': 107.74662,
 '95% HPD interval': array([ 99., 116.]),
 'mc error': 0.024400429422450735,
 'quantiles': {2.5: 99.0,
  5.0: 100.0,
  16.0: 103.0,
  25: 105.0,
  50: 108.0,
  75: 111.0,
  84.0: 112.0,
  95.0: 115.0,
  97.5: 116.0}}

In [34]:
for akey in allStats['d0']['quantiles']:
    print('%10.2f %10.3f %10.3e'%(akey, allStats['d0']['quantiles'][akey],box.EDensity[int(allStats['d0']['quantiles'][akey])]))

      2.50     99.000  2.985e+08
      5.00    100.000  3.162e+08
     16.00    103.000  3.758e+08
     25.00    105.000  4.217e+08
     50.00    108.000  5.012e+08
     75.00    111.000  5.957e+08
     84.00    112.000  6.310e+08
     95.00    115.000  7.499e+08
     97.50    116.000  7.943e+08


### the mean Density index and the mean Density

In [35]:
indexMean = int(allStats['d0']['mean'])
d0Mean = box.EDensity[indexMean]
print(' mean D index %6i  mean D %10.2e'%(indexMean, d0Mean))

 mean D index    107  mean D   4.73e+08


### the 95% high probability density (hpd) interval of the electron density indices and electron density

In [37]:
index95min = allStats['d0']['quantiles'][2.5]
index95max = allStats['d0']['quantiles'][97.5]
d095HpdMin = box.EDensity[int(index95min)]
d095HpdMax = box.EDensity[int(index95max)]
print(' index 2.5  HPD min %7.1f Density %10.2e'%(index95min, d095HpdMin))
print(' index 97.5 HPD max %7.1f Density %10.2e'%(index95max, d095HpdMax))

 index 2.5  HPD min    99.0 Density   2.99e+08
 index 97.5 HPD max   116.0 Density   7.94e+08


In [38]:
d0Std = int(allStats['d0']['standard deviation'])
print(' std of Density index %5i '%(d0Std))

 std of Density index     4 


In [39]:
d0mStd = box.EDensity[indexMean - d0Std]
d0pStd = box.EDensity[indexMean + d0Std]
' density mean -/+ std = %10.2e %10.2e'%(d0mStd,d0pStd)

' density mean -/+ std =   3.76e+08   5.96e+08'

In [40]:
allStats['em0']

{'n': 100000,
 'standard deviation': 0.030618242461250174,
 'mean': 26.954843541207815,
 '95% HPD interval': array([26.89595702, 27.01517192]),
 'mc error': 0.00026331939485677,
 'quantiles': {2.5: 26.891874767982898,
  5.0: 26.902716950975382,
  16.0: 26.92450240949749,
  25: 26.934941951203086,
  50: 26.9559288661834,
  75: 26.975950861414642,
  84.0: 26.98483160343099,
  95.0: 27.003312683457878,
  97.5: 27.011899402302365}}

In [42]:
em0Mean = allStats['em0']['mean']
' std of log10 EM  %10.3f '%(em0Mean)

' std of log10 EM      26.955 '

In [43]:
em0Std = allStats['em0']['standard deviation']
print(' std of log10 EM  %10.3f '%(em0Std))

 std of log10 EM       0.031 


In [44]:
em0mStd = em0Mean - em0Std
em0pStd = em0Mean + em0Std
' density mean -/+ std = %10.4f %10.4f'%(em0mStd,em0pStd)

' density mean -/+ std =    26.9242    26.9855'

In [45]:
em095Hpd = allStats['em0']['95% HPD interval']
print(' 95 HPD interval %12.3f  %12.3f'%(em095Hpd[0],  em095Hpd[1]))

 95 HPD interval       26.896        27.015


In [46]:
itLo = int(allStats['d0']['95% HPD interval'][0])
itHi = int(allStats['d0']['95% HPD interval'][1])
print(' 95 HPD interval %12.3e  %12.3e'%(box.EDensity[itLo],box.EDensity[itHi]))

 95 HPD interval    2.985e+08     7.943e+08


In [47]:
outname = 'tab2_1993_qs_fe_13_mcmc.log'

In [48]:
with open(outname,'w') as outpt:
    outpt.write(' index         = %5i \n'%(indexMean))
    outpt.write(' log EM        = %10.3f \n'%(em0Mean))
    outpt.write(' EM            = %10.3e \n'%(10.**em0Mean))
    outpt.write(' logEM +/- std = %10.3f   %10.3f \n'%(em095Hpd[0], em095Hpd[1]))
    outpt.write(' EM +/- std    = %10.2e   %10.2e \n'%(10.**em095Hpd[0],  10.**em095Hpd[1]))
    outpt.write(' Density       = %10.3e  \n'%(d0Mean))
    outpt.write(' Density +/- std   = %10.2e %10.2e \n'%(d0mStd, d0pStd)) 
    outpt.write(' Density 95pc Hpd  = %10.2e %10.2e \n'%(d095HpdMin, d095HpdMax))

In [49]:
allStats['predictMc1']

{'n': 100000,
 'standard deviation': array([0.3250768 , 1.5521932 , 0.673536  , 0.71005447, 2.30045988,
        0.7774512 , 5.09794163, 2.24513071, 1.33882456]),
 'mean': array([ 2.84849914, 17.54721701,  6.10161187,  8.83744839, 20.1492925 ,
         8.78924009, 43.02788762, 28.99340377, 11.29706245]),
 '95% HPD interval': array([[ 2.2237063 , 14.49778183,  4.77089521,  7.49031139, 15.72806614,
          7.2618454 , 33.22219589, 24.58852382,  8.7219106 ],
        [ 3.48608619, 20.54550798,  7.38780703, 10.27499607, 24.66162365,
         10.29105909, 53.07479402, 33.38349037, 13.93589622]]),
 'mc error': array([0.00212245, 0.01199463, 0.00448274, 0.00546502, 0.01501746,
        0.00600788, 0.03507098, 0.01786975, 0.00920955]),
 'quantiles': {2.5: array([ 2.21297747, 14.51618921,  4.74195184,  7.45738504, 15.65226078,
          7.2710456 , 33.12989329, 24.52917376,  8.69767815]),
  5.0: array([ 2.31425112, 14.97852094,  4.97001442,  7.6881584 , 16.36874767,
          7.50261467, 34.6551

### the samples are stored in  MCMC.pickle

In [50]:
with open('MCMC.pickle','rb') as inpt:
    db = pickle.load(inpt)

In [51]:
db.keys()

dict_keys(['deviance', 'predictMc1', 'em0', 'd0', 'DiscreteMetropolis_d0_adaptive_scale_factor', 'Metropolis_em0_adaptive_scale_factor', '_state_'])

In [52]:
plt.figure()
plt.hist(db['d0'][0],bins=20)

(array([4.0000e+00, 5.0000e+00, 3.8000e+01, 1.1600e+02, 3.0100e+02,
        1.7480e+03, 3.1110e+03, 6.6340e+03, 1.1583e+04, 1.5731e+04,
        2.5532e+04, 1.4527e+04, 1.0596e+04, 5.9880e+03, 2.6670e+03,
        1.1980e+03, 1.6400e+02, 5.0000e+01, 3.0000e+00, 4.0000e+00]),
 array([ 85. ,  87.2,  89.4,  91.6,  93.8,  96. ,  98.2, 100.4, 102.6,
        104.8, 107. , 109.2, 111.4, 113.6, 115.8, 118. , 120.2, 122.4,
        124.6, 126.8, 129. ]),
 <a list of 20 Patch objects>)

In [53]:
plt.figure()
plt.hist(db['d0'][0],bins=np.arange(90,130,1))
xy = plt.axis()
plt.axis([90,130,xy[2],xy[3]])
plt.text(1.e+9, 8000., myIon , fontsize=16)
plt.text(120, 5000., myIon , fontsize=16)
plt.xlabel('Density Index', fontsize=14)
plt.ylabel('Frequency', fontsize=14)
plt.tight_layout()

/usr/lib/python3.6/site-packages/ipykernel_launcher.py:9: UserWarning: Tight layout not applied. The left and right margins cannot be made large enough to accommodate all axes decorations. 
  if __name__ == '__main__':


In [54]:
xyhist=plt.hist(db['d0'][0],bins=np.arange(90,130,1))

In [55]:
plt.figure()
plt.semilogx(box.EDensity[xyhist[1][:-1]], xyhist[0], 'b', lw=3)
xy = plt.axis()
plt.axis([1.e+8,3.e+9,0.,xy[3]])
plt.plot([d0Mean,d0Mean],[xy[2],xy[3]], 'k', lw=2,label='mean')
plt.plot([d0mStd,d0mStd],[xy[2],xy[3]], 'r--', lw=2,label='mean - std')
plt.plot([d0pStd,d0pStd],[xy[2],xy[3]], 'r--', lw=2,label='mean + std')
plt.plot([d095HpdMin,d095HpdMin],[xy[2],xy[3]], 'k--', lw=2,label='95% Hpd Min')
plt.plot([d095HpdMax,d095HpdMax],[xy[2],xy[3]], 'k--', lw=2,label='95% Hpd Max')
plt.xlabel('Electron Density cm$^{-3}$', fontsize=14)
plt.ylabel('Frequency', fontsize=14)
plt.text(1.2e+9, 5000., myIon , fontsize=16)
plt.legend(loc='upper left')
plt.tight_layout()

In [57]:
plt.figure()
box.emPlot(vs='d')
xy = plt.axis()
plt.axis([1.e+6,1.e+12,xy[2],xy[3]])
plt.plot([d0Mean,d0Mean],[xy[2],xy[3]], 'k', lw=2,label='mean')
plt.plot([d0mStd,d0mStd],[xy[2],xy[3]], 'r--', lw=2,label='mean - std')
plt.plot([d0pStd,d0pStd],[xy[2],xy[3]], 'r--', lw=2,label='mean + std')
plt.plot([d095HpdMin,d095HpdMin],[xy[2],xy[3]], 'k--', lw=2,label='95% Hpd Min')
plt.plot([d095HpdMax,d095HpdMax],[xy[2],xy[3]], 'k--', lw=2,label='95% Hpd Max')
plt.legend(loc='lower left')
plt.text(3.e+10, 3.e+26, myIon, fontsize=16)
plt.tight_layout()

In [59]:
len(db['em0'][0])

100000

In [60]:
plt.figure()
plt.hist(db['em0'][0],bins=20)

(array([5.0000e+00, 7.0000e+00, 4.0000e+01, 1.2600e+02, 3.2400e+02,
        6.8000e+02, 1.6490e+03, 3.5170e+03, 6.0680e+03, 1.0036e+04,
        1.3679e+04, 1.6216e+04, 1.6143e+04, 1.3761e+04, 9.1680e+03,
        5.3830e+03, 2.1110e+03, 7.8100e+02, 2.7200e+02, 3.4000e+01]),
 array([26.80485436, 26.81758504, 26.83031572, 26.8430464 , 26.85577708,
        26.86850776, 26.88123844, 26.89396912, 26.90669979, 26.91943047,
        26.93216115, 26.94489183, 26.95762251, 26.97035319, 26.98308387,
        26.99581455, 27.00854523, 27.02127591, 27.03400659, 27.04673726,
        27.05946794]),
 <a list of 20 Patch objects>)

In [61]:
plt.figure()
plt.hist(db['em0'][0],bins=np.arange(26.85,27.1,.01))
xy = plt.axis()
plt.plot([em0Mean, em0Mean],[xy[2],xy[3]], 'k', lw=2,label='mean')
plt.plot([em0mStd, em0mStd],[xy[2],xy[3]], 'r--', lw=2,label='mean - std')
plt.plot([em0pStd, em0pStd],[xy[2],xy[3]], 'r--', lw=2,label='mean + std')
plt.plot([em095Hpd[0], em095Hpd[0]],[xy[2],xy[3]], 'k--', lw=2,label='95% Hpd Min')
plt.plot([em095Hpd[1], em095Hpd[1]],[xy[2],xy[3]], 'k--', lw=2,label='95% Hpd Max')
plt.xlabel('log$_{10}$ Emission Measure (cm$^{-5}$)',fontsize=14)
plt.ylabel('Frequency', fontsize=14)
plt.text(26.86, 11000, 'Fe XIII',fontsize=16)
plt.legend(loc='center left')
plt.tight_layout()

In [62]:
model.summary()


em0:
 
	Mean             SD               MC Error        95% HPD interval
	------------------------------------------------------------------
	26.955           0.031            0.0              [26.896 27.015]
	
	
	Posterior quantiles:
	
	2.5             25              50              75             97.5
	 |---------------|===============|===============|---------------|
	26.892           26.935          26.956         26.976        27.012
	

predictMc1:
 
	Mean             SD               MC Error        95% HPD interval
	------------------------------------------------------------------
	2.848            0.325            0.002              [2.224 3.486]
	17.547           1.552            0.012            [14.498 20.546]
	6.102            0.674            0.004              [4.771 7.388]
	8.837            0.71             0.005            [ 7.49  10.275]
	20.149           2.3              0.015            [15.728 24.662]
	8.789            0.777            0.006            [ 7.262 

In [63]:
emfit = allStats['em0']['mean']
print(' emfit = %10.3f'%(emfit))

 emfit =     26.955


In [64]:
idx1 = int(allStats['d0']['mean'])
print(' Density index = %5i  density = %12.2e'%(idx1,box.EDensity[idx1]))

 Density index =   107  density =     4.73e+08


### use the mean values of the emission measure em0 and the mean value of the density idx1 to predict the line intensities

In [69]:
box.emSetIndices(idx1)

In [70]:
box.emSet(emfit)

In [71]:
box.predict()

In [72]:
box.predictPrint(outfile='predictPrintMc.txt', minContribution=0.01)

   107     4.73e+08     1.78e+06     9.01e+26
 -------------------------------------------------
  iwvl    IonS        wvl        Int       Pred   Int/Pred        chi
               wvl lvl1 lvl2                lower -                upper lineIdx predLine contribution
 -------------------------------------------------
     0   fe_13    311.574   6.13e+00   2.79e+00      2.194      5.486
         fe_13
           311.547    2   12        3s2 3p2 3P1.0 - 3s 3p3 3P2.0            29     0   0.999
 -------------------------------------------------
 -------------------------------------------------
     1   fe_13    312.171   1.78e+01   1.77e+01      1.008      2.521
         fe_13
           312.174    2   11        3s2 3p2 3P1.0 - 3s 3p3 3P1.0            24     0   1.000
 -------------------------------------------------
 -------------------------------------------------
     2   fe_13    312.907   7.34e+00   6.02e+00      1.220      3.051
         fe_13
           312.868    2   10      

In [73]:
box.diffPrintMc()

 index      density  temperature           Em
   107     4.73e+08     1.78e+06     9.01e+26
 -------------------------------------------------
 chi = abs(int/(2*wght*pred))  strDiff = (int - pred)/pred
  iwvl    ionS        wvl  intensity  predicted   int/pred        chi     relDev     stdDev
     0   fe_13    311.574   6.13e+00   2.79e+00      2.194      5.486      0.544      0.544
     1   fe_13    312.171   1.78e+01   1.77e+01      1.008      2.521      0.008      0.008
     2   fe_13    312.907   7.34e+00   6.02e+00      1.220      3.051      0.181      0.181
     3   fe_13    318.129   6.09e+00   8.72e+00      0.698      1.746      0.432      0.432
     4   fe_13    320.802   2.45e+01   1.98e+01      1.240      3.099      0.193      0.193
     5   fe_13    321.464   8.64e+00   8.84e+00      0.977      2.443      0.023      0.023
     6   fe_13    348.196   5.50e+01   4.36e+01      1.262      3.156      0.208      0.208
     7   fe_13    359.658   2.84e+01   2.88e+01      0.986    

### the same information can be obtained from db, so pickling model is not necessary

In [74]:
traces = db['predictMc1'][0]
traces.shape

(100000, 9)

In [84]:
pformat = 'line %2i  wvl:  %8.3f obsInt:  %8.3f  mean predicted %10.3f  absDev %8.3f relDev %8.3f stdDev %6.3f'
wformat = 'line %2i  wvl:  %8.3f obsInt:  %8.3f  mean predicted %10.3f  absDev %8.3f relDev %8.3f stdDev %6.3f \n'
with open('mc_trace.log','w') as outpt:
    relDevList = []
    stdDevList = []
    for iline in range(nobs):
        obsI = box.match[iline]['obsIntensity']
        wvl = box.match[iline]['obsWvl']
        absDev = np.abs(obsI - traces[:,iline].mean())
        relDev = absDev/obsI
        relDevList.append(relDev)
        stdDev = relDev**2
        stdDevList.append(stdDev)
        print(pformat%(iline, wvl, obsI, traces[:,iline].mean(), absDev, relDev,stdDev))
        outpt.write(wformat%(iline, wvl, obsI, traces[:,iline].mean(), absDev, relDev,stdDev))
    relDevMean = np.mean(relDevList)
    stdDevMean = np.mean(stdDevList)
    print(' average relative deviation = %8.3f   3*std = %8.3f stdDev %8.3f'%(relDevMean, 3.*relDevMean, stdDevMean))    
    outpt.write(' average relative deviation = %8.3f   3*std = %8.3f stdDev %8.3f \n'%(relDevMean, 3.*relDevMean, stdDevMean))

line  0  wvl:   311.574 obsInt:     6.130  mean predicted      2.854  absDev    3.276 relDev    0.534 stdDev  0.286
line  1  wvl:   312.171 obsInt:    17.800  mean predicted     17.516  absDev    0.284 relDev    0.016 stdDev  0.000
line  2  wvl:   312.907 obsInt:     7.340  mean predicted      6.098  absDev    1.242 relDev    0.169 stdDev  0.029
line  3  wvl:   318.129 obsInt:     6.090  mean predicted      8.830  absDev    2.740 relDev    0.450 stdDev  0.202
line  4  wvl:   320.802 obsInt:    24.500  mean predicted     20.139  absDev    4.361 relDev    0.178 stdDev  0.032
line  5  wvl:   321.464 obsInt:     8.640  mean predicted      8.772  absDev    0.132 relDev    0.015 stdDev  0.000
line  6  wvl:   348.196 obsInt:    55.000  mean predicted     42.906  absDev   12.094 relDev    0.220 stdDev  0.048
line  7  wvl:   359.658 obsInt:    28.400  mean predicted     28.962  absDev    0.562 relDev    0.020 stdDev  0.000
line  8  wvl:   359.851 obsInt:    11.800  mean predicted     11.269  ab